## Imports

In [ ]:
#import libraries
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import json
from sklearn import metrics

### Util functions

In [ ]:
#Activation Functions
def ReLu(x):
    return 0 if x < 0 else x

def Sigmoid(x):
    return 1/(1+np.exp(-x))

def Tanh(x):
    return np.tanh(x)
    # return (np.exp(x) - np.exp(-x) )/ (np.exp(x) + np.exp(-x))

#Util Functions
def imageToMatrix(img):
    return np.array(img)

def splitToRGBMat(rgb):
    dim = rgb.shape[0]
    r = np.zeros((dim,dim))
    g = np.zeros((dim,dim))
    b = np.zeros((dim,dim))
    for i in range(dim):
        for j in range(dim):
            r[i][j] = rgb[i][j][0]
            g[i][j] = rgb[i][j][1]
            b[i][j] = rgb[i][j][2]
    return r, g, b

def combineRGB(r, g, b):
    res = []
    res.append(r)
    res.append(g)
    res.append(b)
    return np.array(res)

def derive(x):
    return x * (1 - x)

def create_sequences_dataset(data, seq_length):
    dataset = []
    data_len = len(data)
    for i in range(data_len - seq_length):
        seq_end = i + seq_length
        seq_x = data[i:seq_end]
        seq_y = data[seq_end]
        dataset.append([seq_x,seq_y])
    return dataset


### ConvolutionLayer

In [ ]:
class ConvolutionLayer:
    #inp is size of pixels. type -> (length, height). length and height must be same
    #dim is number of dimensions. type -> int. 
    #pad is number of padding, type -> int.
    #filterCtr is number of filter, type -> int
    #filterSize is size of filter, type -> (int, int). Second int is curently unused
    #stride is number of stride, type -> int
    def __init__(self, inp, dim, pad, filterCtr, filterSize, stride, learning):
        # Convolution stage
        self.inp = inp
        self.dim = dim
        self.pad = pad
        self.filterCtr = filterCtr
        self.filterSize = filterSize
        self.stride = stride
        self.filterMat = []

        # Generate filter and bias
        self.genFilter()
        self.bias = [np.random.rand() for i in range(filterCtr)]

        self.tempFilMat = np.zeros(self.filMat.shape)
        self.tempbias = np.zeros(len(self.bias))

        rowCol = self.getResSize(self.inp[0], self.filterSize[0], self.pad, self.stride)
        self.outSize = (rowCol,rowCol,dim)
        self.learning = learning

    def getParamNum(self):
        return self.filterCtr * (self.filterSize[0] * self.filterSize[0] * self.dim + 1) 
    
    def getOutputShape(self):
        return (self.filterCtr, self.getResSize(), self.getResSize())

    def genFilter(self):
        self.filMat = np.full((self.filterCtr, self.dim ,self.filterSize[0], self.filterSize[1]), 0.001)

    def setFilter(self, filt):
        self.filMat = filt

    def setInputMatrix(self, mat):
        self.input_mat = mat

    def addPadding(self):
        paddedShape = (self.inp[0] + (2*self.pad), self.inp[1] + (2*self.pad))
        resMat = []
        for item in self.input_mat:
            paddedMatrix = np.zeros(paddedShape)
            paddedMatrix[self.pad:self.pad + self.inp[0], self.pad:self.pad + self.inp[1]] = item
            resMat.append(paddedMatrix)
        self.input_mat = np.array(resMat)

    def getResSize(self, W, F, P, S):
        return int(((W - F + (2*P))/S) + 1)

    def calcSumMat(self, resMat, filter_index):
        container = np.zeros(resMat[0].shape)

        for item in resMat:
            container += item
        
        container += self.bias[filter_index]

        return container

    def convolution(self):
        resSize = self.getResSize(self.inp[0], self.filterSize[0], self.pad, self.stride)
        resAll = []

        #First shape is how many filters 
        #Second shape is how many channels/dimensions(rgb)
        #Third and fourth shape is filter data
        for filter_index,filmatArr in enumerate(self.filMat): #iterasi filter 
            # resMat = []
            for dim_index,item in enumerate(self.input_mat): #iterasi matrix input
                filMat = filmatArr[dim_index]
                rows,cols = np.shape(item)
                rowsK,colsK = np.shape(filMat)
                floorR = int(np.floor(rowsK/2))
                floorC = int(np.floor(colsK/2))

                res = []
                for i in range(floorR,rows-floorR,self.stride):
                    resRow = []
                    for j in range(floorC,cols-floorC,self.stride):
                        ROI = item[i-floorR:i+floorR+1,j-floorC:j+floorC+1]
                        conv = np.sum(np.multiply(ROI,filMat)) + self.bias[filter_index]
                        resRow.append(conv)
                    res.append(resRow)
                resAll.append(res)
        self.out = np.array(resAll)

    def detector(self):
        detectAll = []

        for item in self.out:
            detect = np.zeros((item.shape[0], item.shape[1]))
            for i in range(item.shape[0]):
                for j in range(item.shape[1]):
                    detect[i][j] = ReLu(item[i][j])
            detectAll.append(detect)

        self.out = np.array(detectAll)

    def backward(self,dE_doutRelu):
        # DETECTOR SECTION 
        dE_doutConv = np.zeros_like(self.out)

        for i in range(self.out.shape[0]):
            for j in range(self.out.shape[1]):
                for k in range(self.out.shape[2]):
                    if self.out[i][j][k] > 0:
                        # If the ReLU output was greater than 0 during the forward pass,
                        # pass the gradient through unchanged
                        dE_doutConv[i][j][k] = dE_doutRelu[i][j][k]
                    else:
                        # If the ReLU output was <= 0, pass zero gradient (no backpropagation)
                        # dE_doutConv[i][j][k] = 0 #not needed, already 0
                        pass

        # CONVOLUTION SECTION
        # -- WEIGHTS
        # dE_dweights = np.zeros_like(self.filMat)  # Initialize with zeros
        dE_dinput = np.zeros_like(self.input_mat, dtype=float)
        learning_rate = self.learning  # TEMPORARY ASSIGNMENT
        resSize = self.getResSize(self.inp[0], self.filterSize[0], self.pad, self.stride)

        for filter_index in range(self.filMat.shape[0]):
            for dim_index in range(self.filMat.shape[1]):
                for i in range(self.filMat.shape[2]):
                    for j in range(self.filMat.shape[3]):

                        # Initialize the sum for this (i, j) element of dE_dweights
                        sum_for_element_ij = 0
                        
                        # Iterate over k and l for this (i, j) element
                        for k in range(resSize):  # resSize is the spatial dimension of the output
                            for l in range(resSize):
                                # Calculate the corresponding input element
                                input_element = self.input_mat[dim_index][i + k*self.stride][j + l*self.stride]
                                
                                # Multiply input_element by the corresponding dE_doutputConv element
                                sum_for_element_ij += input_element * dE_doutConv[filter_index][k][l]

                                # Calculate the gradient contribution for each input element
                                gradient_contribution = self.filMat[filter_index][dim_index][i][j] * dE_doutConv[filter_index][k][l]
                                
                                # Update the corresponding region in dE_dinput
                                dE_dinput[dim_index, i + k*self.stride : i + k*self.stride + self.filterSize[0], 
                                        j + l*self.stride : j + l*self.stride + self.filterSize[1]] += gradient_contribution
                        
                        # Assign the sum to dE_dweights
                        # dE_dweights[filter_index][dim_index][i][j] = sum_for_element_ij
                        self.tempFilMat[filter_index][dim_index][i][j] -= self.learning * sum_for_element_ij # UPDATE WEIGHTS
        # -- BIAS 
        for filter_index in range(self.filMat.shape[0]):
            ass = learning_rate * np.sum(dE_doutConv[filter_index])
            self.tempbias[filter_index] -= self.learning * np.sum(dE_doutConv[filter_index])

    def updateWeight(self, batch_size):
        self.filMat += self.tempFilMat/batch_size
        self.bias += self.tempbias/batch_size
        self.tempFilMat = np.zeros(self.filMat.shape)
        self.tempbias = np.zeros(len(self.bias))


    def forward(self, mat):
        self.setInputMatrix(mat)
        self.addPadding()

        # print("Input matrix: ")
        # print(self.input_mat)
        
        self.convolution()
        # print("Convolution matrix: ")
        # print(self.out)

        self.detector()
        # print("Detector matrix: ")
        # print(self.out)

    def saveLayer(self):
        return{
            "type": "ConvolutionLayer",
            "parameters": {
                "inp": self.inp,
                "dim": self.dim,
                "pad": self.pad,
                "filterCtr": self.filterCtr,
                "filterSize": self.filterSize,
                "stride":self.stride,
                "learning":self.learning 
            },
            "weights": {
                "filMat": self.filMat,
                "bias": self.bias
            }
        }

### PoolingLayer

In [ ]:
class PoolingLayer:
    #poolFilterSzie is size of filter. type -> int
    #poolStride is stride amount. type -> int

    #poolFunction is the function used (max or average). type -> string. 
    #MUST BE EITHER 'AVERAGE' or 'MAX'. Program will return empty string if not.
    
    def __init__(self,poolFilterSize, poolStride, poolFunction, inputShape):
        # Pooling Stage
        self.poolFilterSize = poolFilterSize
        self.poolStride = poolStride
        self.poolFunction = poolFunction
        self.inputShape = inputShape
        # rowCol = self.getPoolSize(inputShape[0],self.poolFilterSize, self.poolStride)
        # self.outSize = (rowCol, rowCol, inputShape[2])
	
    def getParamNum(self):
        return 0
    
    def getOutputShape(self):
        return (self.inputShape[0], self.getPoolSize(), self.getPoolSize()) #how does inputshape work??

    def setInputMatrix(self, mat):
        self.mat = mat

    def getPoolSize(self,W, F, S):
        return int(((W - F)/S) + 1)

    def pooling(self):
        poolSize = self.getPoolSize(self.mat[0].shape[0], self.poolFilterSize, self.poolStride)
        poolAll = []
        max_positions = []  # To store positions of max elements

        for item in self.mat:
            pool = np.zeros((poolSize,poolSize))
            max_positions_item = []  # To store max positions for this item

            for i in range(poolSize):
                for j in range(poolSize):
                    row_start = i * self.poolStride
                    col_start = j * self.poolStride
                    row_end = row_start + self.poolFilterSize
                    col_end = col_start + self.poolFilterSize

                    # Extract the pool slice from the global matrix
                    pool_slice = item[row_start:row_end, col_start:col_end]

                    if self.poolFunction == 'AVERAGE':
                        pool[i][j] = np.mean(pool_slice)
                    elif self.poolFunction == 'MAX':
                        max_val = np.max(pool_slice)
                        pool[i][j] = max_val

                        # Get max position relative to the global matrix
                        max_pos_local = np.unravel_index(np.argmax(pool_slice), pool_slice.shape)
                        max_pos_global = (max_pos_local[0] + row_start, max_pos_local[1] + col_start)
                        max_positions_item.append(max_pos_global)
                        
            poolAll.append(pool)
            max_positions.append(max_positions_item)

        self.out = np.array(poolAll)
        self.max_positions = max_positions

    def backward(self,dEdoutpool):
        if self.poolFunction == 'AVERAGE':
            windowSize = self.poolFilterSize ** 2
            doutpooldoutconv = (1 / windowSize) * np.ones(self.mat.shape)  # Create a matrix of the same shape as the pooled output with constant values
        elif self.poolFunction == 'MAX':
            doutpooldoutconv = np.zeros(self.mat.shape)  # Create a matrix of zeros with the same shape as the pooled output

            # Iterate through the pooled regions and fill in the max positions
            for item_idx, max_positions_item in enumerate(self.max_positions):
                for i in range(dEdoutpool.shape[1]):
                    for j in range(dEdoutpool.shape[2]):
                        max_pos = max_positions_item[i * dEdoutpool.shape[2] + j]
                        doutpooldoutconv[item_idx, max_pos[0], max_pos[1]] = 1  # Set the gradient to 1 at the max position

        # Determine the size of the upsampling based on self.FilterSize
        poolSize =  self.getPoolSize(self.mat[0].shape[0], self.poolFilterSize, self.poolStride)  # Assumes square matrices
        upsampling_factor = self.mat.shape[1] //poolSize

        # Resize dEdoutpool to match the shape of doutpooldoutconv
        dEdoutpool_resized=[]
        for item in dEdoutpool:
            item_temp = np.kron(item, np.ones((upsampling_factor, upsampling_factor)))
            if self.mat.shape[1] % self.poolFilterSize != 0:
                extra_layer = self.mat.shape[1] % poolSize
                item_temp = np.pad(item_temp, ((0,extra_layer),(0,extra_layer)), 'edge')
            dEdoutpool_resized.append(item_temp)
        # Compute the gradient of the loss with respect to the input to the pooling layer
        self.dEdoutconv = dEdoutpool_resized * doutpooldoutconv
        # return dEdoutconv

    def forward(self, mat):
        self.setInputMatrix(mat)
        self.pooling()
        # print("Pooling matrix (results): ")
        # print(self.out)

    def saveLayer(self):
        return {
            "type": "PoolingLayer",
            "parameters": {
                "poolFilterSize": self.poolFilterSize,
                "poolStride": self.poolStride,
                "poolFunction": self.poolFunction,
                "inputShape": self.inputShape
            }
        }

### FlattenLayer

In [ ]:
class FlattenLayer():
    def __init__(self, inputShape):
        self.inputShape = inputShape
        self.units = inputShape[0] * inputShape[1] * inputShape[2]

    def getParamNum(self):
        return 0
    
    def getOutputShape(self):
        return (None, self.units)

    def flatten(self, matrices):
        self.matrices = matrices
        flattened = []
        for matrix in matrices:
            flattened.extend(matrix.ravel())  # Use ravel to flatten
        self.out = np.array(flattened)
        # self.units = len(self.out)

    def forward(self, matrices):
        self.flatten(matrices)
    
    def backward(self, dE_doutflatten):
        self.dEdoutpool = dE_doutflatten.reshape(self.matrices.shape[0],self.matrices.shape[1],self.matrices.shape[2])
        # return dE_doutpool
    
    def saveLayer(self):
        return {
            "type": "FlattenLayer",
            "parameters": {
                "inputShape": self.inputShape
            }
        }

### DenseLayer

In [ ]:
class DenseLayer:
    #units is number of neurons, type -> int
    #activation is activation function, type -> string
    #must be either "sigmoid" or "relu", else will error
    def __init__(self, units, activation, learning, weight = None):
        self.units = units
        self.activation = activation
        self.learning = learning
        self.weight = weight
    
    def info(self):
        if hasattr(self, "weight") and hasattr(self, "net"):
            print("weight:")
            print(self.weight)
            print()

            print("Bias value:")
            print(self.bias)
            print()

            print("net value:")
            print(self.net)
            print()
            

            if hasattr(self, 'activated'):
                print("activated value:")
                print(self.activated)
                print()
        else:
            print("This layer hasn't been initialized")

    def getParamNum(self):
        return (self.input_size + 1) * self.units
    
    def getOutputShape(self):
        return self.units

    def initLayer(self, inpUnits):
        self.input_size = inpUnits
        if not isinstance(self.weight, np.ndarray):
            self.weight = (np.random.rand(self.units, inpUnits + 1) - 0.5) * 2
        self.eror = np.zeros(self.weight.shape)
        self.bias = 1
    
    def forward(self, arr):
        self.arr = np.append(arr, self.bias)
        self.arr = np.transpose(self.arr[np.newaxis])
        self.net = np.matmul(self.weight, self.arr).flatten()
        self.activate()

    def backward(self, target = None, dTotdNetOut = None, weightOut = None, isLast = None):
        # if len(self.net) == 2:
        if isLast:
            dTotdOut = np.subtract(self.out, target)
            if self.activation == 'relu':
                dOutdNet = np.array([0 if x == 0 else 1 for x in self.net])
            elif self.activation == 'sigmoid':
                dOutdNet = np.multiply(self.out, np.subtract(1, self.out)) 
            self.dTotdNet = np.multiply(dTotdOut, dOutdNet)
            self.dTotdInp = np.matmul(np.transpose(self.weight[:,:-1]),np.transpose(self.dTotdNet))

            self.eror += np.transpose(np.matmul(self.arr, [self.dTotdNet]))
            
        else:
            if self.activation == 'relu':
                dOutdNet = np.array([0 if x == 0 else 1 for x in self.net])
            elif self.activation == 'sigmoid':
                dOutdNet = np.multiply(self.out, np.subtract(1, self.out)) 
            dTotdOut = np.matmul(np.transpose(weightOut[:,:-1]),np.transpose(dTotdNetOut))
            self.dTotdNet = np.multiply(dTotdOut, dOutdNet)
            self.dTotdInp = np.matmul(np.transpose(self.weight[:,:-1]),np.transpose(self.dTotdNet))

            self.eror += np.transpose(np.matmul(self.arr, [self.dTotdNet]))
            # self.weight = np.subtract(self.weight, np.multiply(self.learning, self.eror))
            
    def updateWeight(self, batch_size):
        eror_total = self.eror/batch_size
        self.weight = np.subtract(self.weight, np.multiply(self.learning, eror_total))
        self.eror = np.zeros(self.weight.shape)

    def activate(self):
        if self.activation == "relu":
            self.out = np.array([ReLu(x) for x in self.net])
        elif self.activation == "sigmoid":
            self.out = np.array([Sigmoid(x) for x in self.net])

    def saveLayer(self):
        return {
            "type" : "DenseLayer",
            "parameters": {
                "units": self.units,
                "activation": self.activation,
                "learning": self.learning
            },
            "weights": self.weight
        }

### Sequential

In [ ]:
class Sequential:
    def __init__(self, epoch, mini_batch):
        self.layers = []
        self.epoch = epoch
        self.mini_batch = mini_batch

        self.predicted_batch = np.array([])

    def addLayer(self, layer):
        if layer.__class__.__name__ == "DenseLayer":  
            layer.initLayer(self.layers[len(self.layers) - 1].getOutputShape())
        self.layers.append(layer)
        

    def forward(self, inp):
        self.layers[0].forward(inp)
        for i in range(1,len(self.layers)):
            self.layers[i].forward(self.layers[i-1].out)
            if i == len(self.layers) -1:
                return self.layers[i].out

    def backward(self, target):
        for i in range(len(self.layers)-1,-1,-1):
            if self.layers[i].__class__.__name__ == "DenseLayer":
                if i == len(self.layers)-1:
                    self.layers[i].backward(target, isLast = True)
                else:
                    self.layers[i].backward(dTotdNetOut = self.layers[i+1].dTotdNet, weightOut = self.layers[i+1].weight )

            if self.layers[i].__class__.__name__ == "FlattenLayer":
                self.layers[i].backward(self.layers[i+1].dTotdInp)
            
            if self.layers[i].__class__.__name__ == "PoolingLayer":
                self.layers[i].backward(self.layers[i+1].dEdoutpool)

            if self.layers[i].__class__.__name__ == "ConvolutionLayer":
                self.layers[i].backward(self.layers[i+1].dEdoutconv)

            if self.layers[i].__class__.__name__ == "LSTMLayer":
                pass
    
    def updateWeight(self):
        for i in range(len(self.layers)):
            if self.layers[i].__class__.__name__ == "ConvolutionLayer" or self.layers[i].__class__.__name__ == "DenseLayer":
                self.layers[i].updateWeight(self.mini_batch)

    def calcError(self,j, predicted,datasets):
        actual_list = datasets[j][1]
        ret = (1/2) * ((predicted-actual_list)**2)    
        return ret
        ret = 0
        for actual in actual_list:
            ret += (1/2) * ((predicted-actual)**2)    
        return ret    

    def train(self, datasets):
        for e in range(self.epoch):
            print(f"Epoch {int(e) + 1} / {self.epoch}")
            for i in range(0, len(datasets), self.mini_batch):
                print(f"Batch {int(i/self.mini_batch + 1)} / {int(len(datasets) / self.mini_batch)}")
                error = []
                for j in range(i, i+self.mini_batch):
                    inp = datasets[j][0]
                    target = datasets[j][1]
                    predicted = self.forward(inp)
                    self.backward(target)
                    print(predicted)
                    error.append(self.calcError(j, predicted,datasets))
                print("Error: ", np.sum(np.array(error))/self.mini_batch)
                self.updateWeight()
            
    def predict(self, test_data):
        pred = []
        act = []
        for data in test_data:
            out = self.forward(data[0])
            pred.append(out)
            act.append(data[1])
        pred1 = [int(np.round(x)) for x in pred]
        confusion_matrix = metrics.confusion_matrix(act, pred1)
        print("Confusion Matrix: ")
        print(confusion_matrix)

    def convert_numpy_to_list(self,data):
        if isinstance(data, np.ndarray):
            return data.tolist()
        elif isinstance(data, dict):
            return {key: self.convert_numpy_to_list(value) for key, value in data.items()}
        elif isinstance(data, list):
            return [self.convert_numpy_to_list(item) for item in data]
        else:
            return data
            
    def saveModel(self, filename):
        json_model = []
        for layer in self.layers:
            json_layer = layer.saveLayer()
            json_layer = self.convert_numpy_to_list(json_layer)
            json_model.append(json_layer)
        
        print(json_model)

        with open(filename, 'w') as json_file:
            json.dump(json_model, json_file, indent=4)

    def loadModel(self, filename):
        file_path = filename

        with open(file_path, 'r') as json_file:
            layers = json.load(json_file)
        
        for layerjson in layers:
            if layerjson == None:
                continue
            layertype = layerjson["type"]
            if layertype == "ConvolutionLayer":
                layer = ConvolutionLayer(
                    layerjson['parameters']['inp'],
                    layerjson['parameters']['dim'],
                    layerjson['parameters']['pad'],
                    layerjson['parameters']['filterCtr'],
                    layerjson['parameters']['filterSize'],
                    layerjson['parameters']['stride'], 
                    layerjson['parameters']['learning'],
                )
                layer.filMat =  layerjson['weights']['filMat'][0],
                layer.bias = layerjson['weights']['bias']
            elif(layertype == "PoolingLayer"):
                layer = PoolingLayer(
                    layerjson['parameters']['poolFilterSize'],
                    layerjson['parameters']['poolStride'],
                    layerjson['parameters']['poolFunction'],
                    layerjson['parameters']['inputShape']
                )
            elif(layertype == "FlattenLayer"):
                layer = FlattenLayer(
                    layerjson['parameters']['inputShape']
                    )
            elif(layertype == "DenseLayer"):
                layer = DenseLayer(
                    layerjson['parameters']['units'],
                    layerjson['parameters']['activation'],
                    layerjson['parameters']['learning'],
                    np.array(layerjson['weights'])
                    )
            elif(layertype == "LSTMLayer"):
                layer = LSTMLayer(
                    layerjson['parameters']['num_timestep'],
                    layerjson['parameters']['input_size'],
                    layerjson['parameters']['num_cell']
                )
                layer.setWeightsBias(
                    layerjson['weights'],
                    layerjson['bias']
                )
           
            self.addLayer(layer)

    def info(self):
        name =  "Layer Type"
        shape = "Output Shape"
        param = "Params" 
        totalparam=0
        print(f"{name:<20} {shape:<20} {param:<20}")
        print( "================================================")
        for layer in self.layers:
            name = layer.__class__.__name__
            shape = str(layer.getOutputShape())
            param = str(layer.getParamNum())
            totalparam+= int(param)
            print(f"{name:<20} {shape:<20} {param:<20}")
        print( "================================================")
        print(f"Total Parameters: {totalparam}")

### LSTMLayer

In [ ]:
class LSTMLayer:
    def __init__(self,num_timestep:int, input_size: int, num_cell: int ):
        self.num_timestep = num_timestep
        self.input_size = input_size

        self.num_cell = num_cell
        self.initGatesStates()
        self.initWeightsBias()

    def getParamNum(self):
        return (self.input_size + self.num_cell + 1) * 4 * self.num_cell 
    
    def getOutputShape(self):
        return self.num_cell

    def initWeightsBias(self):
        inp = self.input_size 
        num = self.num_cell

        random_scale = 0.1

        # For weight matrices
        self.U_f = (2 * np.random.rand(num, inp) - 1) * random_scale
        self.U_i = (2 * np.random.rand(num, inp) - 1) * random_scale
        self.U_c = (2 * np.random.rand(num, inp) - 1) * random_scale
        self.U_o = (2 * np.random.rand(num, inp) - 1) * random_scale

        self.W_f = (2 * np.random.rand(num, num) - 1) * random_scale
        self.W_i = (2 * np.random.rand(num, num) - 1) * random_scale
        self.W_c = (2 * np.random.rand(num, num) - 1) * random_scale
        self.W_o = (2 * np.random.rand(num, num) - 1) * random_scale

        # For bias vectors
        self.b_f = (2 * np.random.rand(num) - 1) * random_scale
        self.b_i = (2 * np.random.rand(num) - 1) * random_scale
        self.b_c = (2 * np.random.rand(num) - 1) * random_scale
        self.b_o = (2 * np.random.rand(num) - 1) * random_scale

    def setWeightsBias(self,weights,bias):
        inp = self.input_size  #size of input_shape[1]
        num = self.num_cell

        self.U_f = weights["U_f"]
        self.U_i = weights["U_i"]
        self.U_o = weights["U_o"]
        self.U_c = weights["U_c"]             

        self.W_f = weights["W_f"]
        self.W_i = weights["W_i"]
        self.W_c = weights["W_c"]
        self.W_o = weights["W_o"]

        self.b_f = bias["b_f"]
        self.b_i = bias["b_i"]
        self.b_c = bias["b_c"]
        self.b_o = bias["b_o"]

    def setInputMatrix(self, mat):
        if mat.shape != (self.num_timestep,self.input_size):
            raise ValueError("INPUT SIZE INVALID")
        self.input_mat = mat

    def initGatesStates(self):
        #IMPORTANT: TIMESTEP STARTS AT 1
        #1 IS USED FOR H-1,C-1 SHENANIGANS
        #sebenernya f_gate etc gaperlu +1 sih , tapi males ganti

        time = self.num_timestep + 1
        num = self.num_cell

        self.f_gate = np.zeros((time, num))
        self.i_gate = np.zeros((time, num))
        self.c_gate = np.zeros((time, num))
        self.o_gate = np.zeros((time, num))

        self.cell_state = np.zeros((time, num))
        self.h_state = np.zeros((time, num))

    def forward(self, input_mat):
        self.setInputMatrix(input_mat)

        #IMPORTANT: TIMESTEP STARTS AT 1
        #0 IS USED FOR H-1,C-1 SHENANIGANS
        for t_idx, input_arr in enumerate(self.input_mat):
            t_idx+=1 #timestep index

            #f 
            net_f = np.matmul(self.U_f, input_arr) + np.matmul(self.W_f, self.h_state[t_idx - 1]) + self.b_f
            self.f_gate[t_idx] = Sigmoid(net_f)

            #i
            net_i = np.matmul(self.U_i, input_arr) + np.matmul(self.W_i, self.h_state[t_idx - 1]) + self.b_i
            self.i_gate[t_idx] = Sigmoid(net_i)

            #c
            net_c = np.matmul(self.U_c, input_arr) + np.matmul(self.W_c, self.h_state[t_idx - 1]) + self.b_c
            self.c_gate[t_idx] = Tanh(net_c)

            #o
            net_o = np.matmul(self.U_o, input_arr) + np.matmul(self.W_o, self.h_state[t_idx - 1]) + self.b_o
            self.o_gate[t_idx] = Sigmoid(net_o)

            #CellState
            self.cell_state[t_idx] = self.f_gate[t_idx] * self.cell_state[t_idx-1] + self.i_gate[t_idx] * self.c_gate[t_idx]

            #HiddenState
            self.h_state[t_idx] = self.o_gate[t_idx] * Tanh(self.cell_state[t_idx]) 
        
        self.out = self.h_state[-1]


    def backward(self):
        pass
          
    def updateWeight(self, batch_size):
        pass

    def saveLayer(self):
        return {
            "type" : "LSTMLayer",
            "parameters": {
                "num_timestep": self.num_timestep,
                "input_size": self.input_size,
                "num_cell": self.num_cell,
            },
            "weights": {
                "U_f":self.U_f,
                "U_i":self.U_i,
                "U_c":self.U_c,
                "U_o":self.U_o,

                "W_f":self.W_f,
                "W_i":self.W_i,
                "W_c":self.W_c,
                "W_o":self.W_o,
            },
            "bias":{
                "b_f":self.b_f,
                "b_i":self.b_i,
                "b_c":self.b_c,
                "b_o":self.b_o,
            }
        }


### Testing

In [ ]:
# Initialize LSTMLayer
input_shape = (2, 2)  # 10 time steps, 4 features per time step
num_cell = 1  # Number of LSTM cells

lstm_layer = LSTMLayer(input_shape[0],input_shape[1], num_cell)

input_data = np.array([
    [0.5, 3],
    [1,   2]
])

weights = {
    'U_f': [[0.5, 0.75]], 
    'U_i': [[0.81, 0.2]], 
    'U_c': [[0.35, 0.45]], 
    'U_o': [[0.4, 0.6]], 

    'W_f': [[0.3]], 
    'W_i': [[0.7]], 
    'W_c': [[0.35]],
    'W_o': [[0.4]]
} 
bias = {
    'b_f': [0.4], 
    'b_i': [0.55], 
    'b_c': [0.25],
    'b_o': [0.5]
    }

lstm_layer.setWeightsBias(weights,bias)

# Perform the forward pass
lstm_layer.forward(input_data)

# Print the results or do further testing as needed
print("f_gate:", lstm_layer.f_gate)
print("i_gate:", lstm_layer.i_gate)
print("c_gate:", lstm_layer.c_gate)
print("o_gate:", lstm_layer.o_gate)
print("cell_state:", lstm_layer.cell_state)
print("h_state:", lstm_layer.h_state)

print(lstm_layer.saveLayer())

f_gate: [[0.        ]
 [0.94784644]
 [0.92962105]]
i_gate: [[0.        ]
 [0.82563471]
 [0.89862687]]
c_gate: [[0.        ]
 [0.94415485]
 [0.93677378]]
o_gate: [[0.        ]
 [0.92414182]
 [0.91223037]]
cell_state: [[0.        ]
 [0.77952702]
 [1.56647482]]
h_state: [[0.        ]
 [0.6029426 ]
 [0.83602558]]
{'type': 'LSTMLayer', 'parameters': {'num_timestep': 2, 'input_size': 2, 'num_cell': 1}, 'weights': {'U_f': [[0.5, 0.75]], 'U_i': [[0.81, 0.2]], 'U_c': [[0.35, 0.45]], 'U_o': [[0.4, 0.6]], 'W_f': [[0.3]], 'W_i': [[0.7]], 'W_c': [[0.35]], 'W_o': [[0.4]]}, 'bias': {'b_f': [0.4], 'b_i': [0.55], 'b_c': [0.25], 'b_o': [0.5]}}


### Data Setup

In [ ]:
trainStockMarket = "Dataset/Dataset_Tubes_LSTM/Train_stock_market.csv"
testStockMarket = "Dataset/Dataset_Tubes_LSTM/Test_stock_market.csv"

df = pd.read_csv(trainStockMarket)
df

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,07-09-1984,5.25,5.500,7900,5.50,5.25,5.25
1,10-09-1984,5.25,5.250,600,5.50,5.25,5.25
2,11-09-1984,5.25,5.250,3500,5.50,5.25,5.25
3,12-09-1984,5.50,5.500,700,5.50,5.50,5.50
4,13-09-1984,5.00,5.500,1700,5.50,5.00,5.00
...,...,...,...,...,...,...,...
9640,06-12-2022,3.76,3.800,22400,3.99,3.81,3.81
9641,07-12-2022,3.68,3.750,18000,3.85,3.74,3.74
9642,08-12-2022,3.80,3.820,51600,4.00,3.85,3.85
9643,09-12-2022,3.85,3.930,7800,3.93,3.88,3.88


In [ ]:
merged_data = df[['Low','Open','Volume','High','Close']].values

### Experiment

In [ ]:
nn = Sequential(epoch=1,mini_batch=5)

lstm = LSTMLayer(num_timestep=4, input_size=5, num_cell=64)
Dense1 = DenseLayer(units=5, activation='sigmoid',learning=0.5)

nn.addLayer(lstm)
nn.addLayer(Dense1)

nn.info()

Layer Type           Output Shape         Params              
LSTMLayer            64                   17920               
DenseLayer           5                    325                 
Total Parameters: 18245


In [ ]:
nn.saveModel("LSTM1.json")

[{'type': 'LSTMLayer', 'parameters': {'num_timestep': 4, 'input_size': 5, 'num_cell': 64}, 'weights': {'U_f': [[2.472182538186818e-05, -0.07937242120534055, 0.07273403170797073, 0.09055178847187276, 0.04139309809412646], [-0.01998503018392286, -0.05405061589832086, -0.005720625840760208, 0.08123513279532138, 0.095123763692675], [0.058599635555450585, -0.045308826621299494, -0.03659296830441166, -0.019596433137722857, -0.0725026902511728], [-0.060964016115986924, 0.05119697881166747, 0.03280946975377035, 0.08562647222699869, 0.03920034434542661], [-0.016829395365116562, -0.01796683772716734, 0.07471995085547832, 0.028622762348408215, -0.07046675460809107], [-0.09254364599341852, -0.02243328260301991, 0.05875528273285105, 0.040623603131641865, 0.021635212550484263], [-0.09245997112274894, 0.0057165103026754595, 0.044160577243776804, -0.07513392758676823, 0.07170022337883981], [-0.0063003064744814544, 0.03563593312990272, 0.027530670088365097, 0.014795085915391804, -0.056332530832932375],

In [ ]:
nn2 = Sequential(1,5)
nn2.loadModel("LSTM1.json")
nn2.info()

Layer Type           Output Shape         Params              
LSTMLayer            64                   17920               
DenseLayer           5                    325                 
Total Parameters: 18245


In [ ]:
nn3 = Sequential(epoch=1,mini_batch=5)

lstm = LSTMLayer(num_timestep=2, input_size=5, num_cell=64)
Dense1 = DenseLayer(units=5, activation='sigmoid',learning= 0.5)

nn3.addLayer(lstm)
nn3.addLayer(Dense1)

nn3.info()

Layer Type           Output Shape         Params              
LSTMLayer            64                   17920               
DenseLayer           5                    325                 
Total Parameters: 18245


In [ ]:
nn4 = Sequential(epoch=1,mini_batch=5)

lstm = LSTMLayer(num_timestep=7, input_size=5, num_cell=64)
Dense1 = DenseLayer(units=5, activation='sigmoid',learning= 0.5)

nn4.addLayer(lstm)
nn4.addLayer(Dense1)

nn4.info()

Layer Type           Output Shape         Params              
LSTMLayer            64                   17920               
DenseLayer           5                    325                 
Total Parameters: 18245


In [ ]:
dataset2 = create_sequences_dataset(merged_data,4)
dataset3 = create_sequences_dataset(merged_data,2)
dataset4 = create_sequences_dataset(merged_data,7)

In [ ]:
predicted_data2 = []
for (data,_) in dataset2:
    prediction = nn.forward(data)
    predicted_data2.append(prediction)

predicted_data2 

/tmp/ipykernel_132/3379207600.py:6: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


[array([0.85218166, 0.85912889, 0.48562015, 0.10395347, 0.16195219]),
 array([0.83926817, 0.85301794, 0.45513369, 0.10659532, 0.15269394]),
 array([0.83730003, 0.85266752, 0.44238644, 0.11494859, 0.15854952]),
 array([0.83658267, 0.85258306, 0.43903317, 0.1169137 , 0.15955   ]),
 array([0.83554749, 0.85240828, 0.43007435, 0.12402292, 0.16488474]),
 array([0.86781726, 0.88807165, 0.5694343 , 0.08364289, 0.15969533]),
 array([0.82375905, 0.8518473 , 0.43182596, 0.10247017, 0.13211627]),
 array([0.82825663, 0.85223721, 0.44331737, 0.09938394, 0.13367023]),
 array([0.82513429, 0.85189085, 0.43043831, 0.10581673, 0.13652188]),
 array([0.83553806, 0.8524119 , 0.42997895, 0.12411861, 0.16491905]),
 array([0.83541107, 0.85240002, 0.42961398, 0.12425602, 0.16489019]),
 array([0.83761953, 0.85260817, 0.43600329, 0.12187581, 0.16539328]),
 array([0.83319963, 0.85219355, 0.42330725, 0.12665325, 0.1643939 ]),
 array([0.83797597, 0.85264137, 0.43704265, 0.1214913 , 0.16547954]),
 array([0.83971877, 

In [ ]:
predicted_data3 = []
for (data,_) in dataset3:
    prediction = nn3.forward(data)
    predicted_data3.append(prediction)

predicted_data3 

/tmp/ipykernel_132/3379207600.py:6: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


[array([0.83096473, 0.3544509 , 0.18162076, 0.50725105, 0.79576588]),
 array([0.79249363, 0.28979186, 0.18251972, 0.49560502, 0.77195364]),
 array([0.82645514, 0.34547029, 0.1821167 , 0.50683817, 0.79295352]),
 array([0.7978179 , 0.29827864, 0.18325414, 0.49801266, 0.77511127]),
 array([0.79660901, 0.28895805, 0.18372697, 0.49824433, 0.77303716]),
 array([0.79835614, 0.29002197, 0.18323906, 0.49848592, 0.77384524]),
 array([0.79913811, 0.29044052, 0.1829736 , 0.49855748, 0.77419398]),
 array([0.8329316 , 0.37286063, 0.19288494, 0.49908656, 0.80117004]),
 array([0.78637704, 0.29689149, 0.18444993, 0.49450837, 0.78214287]),
 array([0.7999198 , 0.29115028, 0.18279065, 0.49868943, 0.77459699]),
 array([0.79957951, 0.29075607, 0.18284601, 0.49861427, 0.7744057 ]),
 array([0.80027604, 0.29126659, 0.1826476 , 0.49870642, 0.77474229]),
 array([0.79978167, 0.29090589, 0.18278902, 0.4986414 , 0.77450368]),
 array([0.79946092, 0.2908334 , 0.18292682, 0.49863278, 0.77437893]),
 array([0.79964012, 

In [ ]:
predicted_data4 = []
for (data,_) in dataset4:
    prediction = nn4.forward(data)
    predicted_data4.append(prediction)

predicted_data4 

/tmp/ipykernel_132/3379207600.py:6: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


[array([0.01697855, 0.09918238, 0.23109454, 0.85118677, 0.02228673]),
 array([0.01678952, 0.09892222, 0.23119275, 0.85194037, 0.02230177]),
 array([0.04421295, 0.17230127, 0.35574193, 0.84608338, 0.02477957]),
 array([0.01612846, 0.10383888, 0.24748674, 0.84375431, 0.02253283]),
 array([0.01754053, 0.09533687, 0.21858661, 0.85788901, 0.02211236]),
 array([0.01751958, 0.09373238, 0.21431873, 0.86086366, 0.02206338]),
 array([0.01754918, 0.09371341, 0.21413057, 0.8607852 , 0.0220565 ]),
 array([0.01752945, 0.0937841 , 0.21439919, 0.86069743, 0.02206148]),
 array([0.01775209, 0.09421509, 0.21502024, 0.86022784, 0.02206956]),
 array([0.02058337, 0.0765117 , 0.15921706, 0.88973389, 0.02117605]),
 array([0.02107632, 0.07730594, 0.16047205, 0.88888305, 0.02119851]),
 array([0.0217716 , 0.0788602 , 0.16371993, 0.88794543, 0.02128678]),
 array([0.03066947, 0.10259087, 0.21940831, 0.87889309, 0.02278349]),
 array([0.0206103 , 0.0811874 , 0.17193104, 0.88268852, 0.02139776]),
 array([0.01995333, 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6fb2877e-0ffc-49c6-bf97-4def005ce532' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>